In [1]:
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import time
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib

/home/veruska/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data_path = '../../data/output/normalized_data_X1.csv'
y_path = '../../data/output/y1.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(1432633, 420)
(1432633, 1)


route   tripNum  shapeSequence  shapeLat  shapeLon  \
0  0.118068  0.066667       0.085019  0.264600  0.496150   
1  0.118068  0.066667       0.085026  0.270218  0.501802   
2  0.118068  0.066667       0.085033  0.273653  0.505296   
3  0.118068  0.066667       0.085039  0.277433  0.509404   
4  0.118068  0.066667       0.085053  0.280050  0.513166   

   distanceTraveledShape  busCode  gpsPointId    gpsLat    gpsLon  ...  \
0               0.057584      0.0    0.000000  0.266155  0.496047  ...   
1               0.064231      0.0    0.000148  0.271812  0.501642  ...   
2               0.068301      0.0    0.000000  0.275187  0.505192  ...   
3               0.072900      0.0    0.000149  0.280622  0.510132  ...   
4               0.077983      0.0    0.000155  0.281878  0.513178  ...   

   alertTypeSB_ACCIDENT  alertTypeSB_CHIT_CHAT  alertTypeSB_HAZARD  \
0                   0.0                    0.0                 0.0   
1                   0.0                    0.0                 0.0   
2                   0.0                    0.0                 0.0   
3                   0.0                    0.0                 0.0   
4                   0.0                    0.0                 0.0   

   alertTypeSB_JAM  alertTypeSB_NORMAL  alertTypeSB_POLICE  \
0              1.0                 0.0                 0.0   
1              0.0                 1.0                 0.0   
2              1.0                 0.0                 0.0   
3              0.0                 1.0                 0.0   
4              1.0                 0.0                 0.0   

   alertTypeSB_ROAD_CLOSED  jamBlockTypeSB_-  jamBlockTypeSB_NORMAL  \
0                      0.0               1.0                    0.0   
1                      0.0               0.0                    1.0   
2                      0.0               1.0                    0.0   
3                      0.0               1.0                    0.0   
4                      0.0               1.0                    0.0   

   jamBlockTypeSB_ROAD_CLOSED_EVENT  
0                               0.0  
1                               0.0  
2                               0.0  
3                               0.0  
4                               0.0  

[5 rows x 420 columns]

In [3]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y, test_size=0.20, random_state=7)

In [4]:
del y
del data

In [ ]:
random.seed(42)

start = time.time()

parameters = {
    'max_depth': [5, 25],
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.01]
}

xgb = XGBClassifier(
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 #nthread=-1,
 scale_pos_weight=1,
 seed=23)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=parameters,
    scoring = 'roc_auc',
    cv = 10
)

grid_search.fit(train_X, train_Y)

end = time.time()
print("Execution time: " + str((end - start)/60) + " min")

In [5]:
xgb = XGBClassifier(
 learning_rate =0.1,
 n_estimators=120, 
 max_depth=50,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=-1,
 scale_pos_weight=1,
 seed=23)

xgb.fit(train_X, train_Y)

/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/veruska/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=120, n_jobs=1,
              nthread=-1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=23,
              silent=None, subsample=0.8, verbosity=1)

In [10]:
# Saving a pickle file for the mode
joblib.dump(xgb, 'Saved_XGB_BB_class.pkl')

['Saved_XGB_BB_class.pkl']

In [6]:
pred_array = xgb.predict(test_X)

In [7]:
pred_array

array([0., 1., 0., ..., 0., 0., 0.])

In [8]:
# Bus Bunching
accuracy = accuracy_score(test_Y, pred_array)
precision = precision_score(test_Y, pred_array)
recall = recall_score(test_Y, pred_array)
f_measure = f1_score(test_Y, pred_array)

In [9]:
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F-measure: " + str(f_measure))

Accuracy: 0.9502525067445651
Precision: 0.9080629466832199
Recall: 0.7015320009331986
F-measure: 0.7915472360339281
